# Functions

In [5]:
#=
Attempt at a "1D" model in order to provide a reasonable starting point for design/optimization

Goal for option 1: input open rotor design, output duct inlet and outlet areas
Goal for option 2: input some performance metric (say, thrust), and some sizing guide (say rotor diameter), and some operating conditions (say, Vinf and Omega), and output reasonable duct inlet and outlet as well as some rotor geometry (chord, twist, camber?)

=#

import FLOWMath as fm

######################################################################
#                                                                    #
#                         1D Model Option 1                          #
#                                                                    #
######################################################################

"""

**Arguments:**
- `thrust::Float` : desired thrust force.
- `exit_area::Float` : exit area of nozzle
- `Vinf::Float` : freestream velocity
- `Rtip::Float` : rotor tip radius
- `Rhub::Float` : rotor hub radius
- `radii:Vector{Float}` : radial positions along the blade (dimensional)
- `num_blades::Int` : number of blades
- `lift_polars::Vector{Matrix{Float}}` : matrix [α cℓ] of angles of attack vs lift coefficients for desired airfoils
-

**Keyword Arguments:**
- `rho::Float` : air density (ρ), assumed constant = 1.225 kg/m3
- `ambient_static_pressure::Float` : ambient static pressure, default = 101325.0 Pa
- `ambient_static_temperature::Float` : ambient static temperature, default = 288.15 K
- `flow_coeff::Float` : tip flow coefficient (ϕ), default = 0.4
- `adiabatic_stage_efficiency::Float` : adiabatic stage efficiency, default = 0.85
- `c_p::Float` : specific heat at constant pressure, default is 1005 J/kg-K
- `lift_coefficients::Vector{Float}` : desired operating lift coefficient for each blade section, default = [0.6] (constant along blade)
- `verbose::Bool` : if true, will print out warning if solidity exceeds 0.3
- `debug::Bool` : if true, function returns intermediate values in a named tuple

**Returns:**
- `chords::Vector{Float}` : vector of chord distribution
- `twists::Vector{Float}` : vector of twist distribution
- `debug::NamedTuple` : NamedTuple of intermediate values
"""
function opt_prelim(
    Vinf,
    thrust,
    exit_area,
    Rtip,
    Rhub,
    radii,
    num_blades,
    lift_polars;
    rho=1.225,
    flow_coeff=0.4,
    ambient_static_pressure=101325.0,
    ambient_static_temperature=288.15,
    adiabatic_stage_efficiency=0.85,
    c_p=1005.0,
    lift_coefficients=[0.6],
    verbose=true,
)

    # Compute Exit Velocity m/s
    Vexit = compute_exit_velocity(Vinf, thrust, exit_area; rho=rho)

    # Compute Fan Face Axial Velocity m/s
    fan_face_axial_velocity = compute_fan_face_axial_velocity(Vexit, exit_area, Rtip, Rhub)

    # Compute Tip Rotational Velocity m/s
    tip_rotational_velocity = compute_tip_rotational_velocity(
        fan_face_axial_velocity; flow_coeff=flow_coeff
    )

    # Compute Exhaust Total Pressue Pa = N/m2 = kg/m-s2
    exhaust_total_pressure = compute_exhaust_total_pressure(
        Vexit; rho=rho, ambient_static_pressure=ambient_static_pressure
    )

    # Compute the Fan Pressure Ratio (non-dimensional)
    pressure_ratio = compute_fan_pressure_ratio(
        exhaust_total_pressure,
        Vinf;
        rho=rho,
        ambient_static_pressure=ambient_static_pressure,
    )

    # Compute_fan_temperature_ratio (non-dimensional)
    temperature_ratio = compute_fan_temperature_ratio(
        pressure_ratio; gamma=1.4, adiabatic_stage_efficiency=adiabatic_stage_efficiency
    )

    # Compute Total Specific Enthalpy Rise J/kg
    enthalpy_rise = compute_total_specific_enthalpy_rise(
        temperature_ratio,
        Vinf;
        c_p=c_p,
        ambient_static_temperature=ambient_static_temperature,
    )

    # Compute the Required Work Coefficient (non-dimensional)
    work_coeff = compute_work_coefficient(enthalpy_rise, tip_rotational_velocity)

    # Compute Change in Swirl Velocity at Tip m/s
    change_in_tip_swirl = compute_change_in_swirl(work_coeff, tip_rotational_velocity)

    # Compute Inflow Angles radians
    inflow_angles = compute_inflow_angles(
        fan_face_axial_velocity, tip_rotational_velocity, Rtip, radii
    )

    # Compute Swirl Velocity Distribution m/s
    swirl_velocities = compute_swirl_velocity_distribution(Rtip, radii, change_in_tip_swirl)

    # Look up Section Angles of Attack radians
    angles_of_attack = look_up_angle_of_attack(
        lift_polars, lift_coefficients, length(radii)
    )

    # Calculate Stagger Angles radians
    stagger_angles = calculate_stagger_angles(inflow_angles, angles_of_attack)

    # Calculate Circulation Distribution m2/s
    circulations = calculate_circulation(swirl_velocities, radii, num_blades)

    # Calculate Chord m
    chords = calculate_chord(
        circulations, lift_coefficients, fan_face_axial_velocity, swirl_velocities
    )

    # Check Solidity (non-dimensional)
    solidity = check_solidity(chords, radii, num_blades, verbose)

    return chords,
    pi / 2.0 .- stagger_angles,
    (;
        solidity,
        circulations,
        stagger_angles,
        angles_of_attack,
        swirl_velocities,
        inflow_angles,
        omega=compute_omega(tip_rotational_velocity, Rtip),
        change_in_tip_swirl,
        work_coeff,
        enthalpy_rise,
        temperature_ratio,
        pressure_ratio,
        exhaust_total_pressure,
        tip_rotational_velocity,
        fan_face_axial_velocity,
        fan_annulus_area=compute_annulus_area(Rtip, Rhub),
        Vexit,
    )
end

"""

Apply conservation of mass and momentum to obtain exit velocity.
"""
function compute_exit_velocity(Vinf, thrust, exit_area; rho=1.225)
    return 0.5*(Vinf +  sqrt(Vinf^2 + 4.0 * thrust / (rho * exit_area)))
end

"""
"""
function compute_annulus_area(Rtip, Rhub)
    return pi * (Rtip^2 - Rhub^2)
end

"""
"""
function compute_fan_face_axial_velocity(Vexit, exit_area, Rtip, Rhub)

    # Get fan annulus area
    fan_area = compute_annulus_area(Rtip, Rhub)

    # convervation of mass
    return Vexit * exit_area / fan_area
end

"""
"""
function compute_tip_rotational_velocity(Cz; flow_coeff=0.4)
    return Cz / flow_coeff
end

"""
"""
function compute_exhaust_total_pressure(Vexit; rho=1.225, ambient_static_pressure=101325.0)
    return 0.5 * rho * Vexit^2 + ambient_static_pressure
end

"""
"""
function compute_fan_pressure_ratio(pte, Vinf; rho=1.225, ambient_static_pressure=101325.0)
    ptinf = 0.5 * rho * Vinf^2 + ambient_static_pressure

    return pte / ptinf
end

"""
"""
function compute_fan_temperature_ratio(
    pressure_ratio; gamma=1.4, adiabatic_stage_efficiency=0.85
)
    return (pressure_ratio^((gamma - 1) / gamma) - 1.0) / adiabatic_stage_efficiency + 1.0
end

"""
"""
function compute_total_specific_enthalpy_rise(
    temperature_ratio, Vinf; c_p=1.005, ambient_static_temperature=288.15
)
    return c_p * (temperature_ratio - 1.0) * ambient_static_temperature
end

"""
"""
function compute_work_coefficient(enthalpy_rise, tip_rotational_velocity)
    return enthalpy_rise / tip_rotational_velocity^2
end

"""
"""
function compute_change_in_swirl(work_coeff, tip_rotational_velocity)
    return work_coeff * tip_rotational_velocity
end

"""
"""
function compute_omega(tip_rotational_velocity, Rtip)
    return (tip_rotational_velocity / Rtip)
end

"""

radians
"""
function compute_inflow_angles(
    fan_face_axial_velocity, tip_rotational_velocity, Rtip, radii
)
    omega = compute_omega(tip_rotational_velocity, Rtip)

    return atan.(omega * radii, fan_face_axial_velocity)
end

"""
"""
function compute_swirl_velocity_distribution(Rtip, radii, change_in_tip_swirl)
    return Rtip * change_in_tip_swirl ./ radii
end

"""
"""
function look_up_angle_of_attack(lift_polars, lift_coefficients, ns)

    # Initialize Output
    angles_of_attack = zeros(eltype(lift_coefficients), length(lift_coefficients))
    # Loop through sections
    for (is, cl) in enumerate(lift_coefficients)

        # get entry just before
        i1 = searchsortedlast(lift_polars[is][:, 2], cl)

        # get entry just after
        i2 = searchsortedfirst(lift_polars[is][:, 2], cl)

        # interpolate
        angles_of_attack[is] = fm.linear(
            [lift_polars[is][i1, 2]; lift_polars[is][i2, 2]],
            [lift_polars[is][i1, 1]; lift_polars[is][i2, 1]],
            cl,
        )
    end

    if length(angles_of_attack) == 1
        return fill(angles_of_attack[1], ns)
    else
        return angles_of_attack
    end
end

"""
"""
function calculate_stagger_angles(inflow_angles, angles_of_attack)
    return inflow_angles .- angles_of_attack
end

"""
"""
function calculate_circulation(swirl_velocities, radii, num_blades)
    return 2.0 * pi * swirl_velocities .* radii / num_blades
end

"""
"""
function calculate_chord(
    circulations, lift_coefficients, fan_face_axial_velocity, swirl_velocities
)
    average_relative_velocity =
        sqrt.(fan_face_axial_velocity^2 .+ (swirl_velocities / 2.0) .^ 2)

    return 2.0 * circulations ./ (lift_coefficients .* average_relative_velocity)
end

function check_solidity(chords, radii, num_blades, verbose)
    solidity = chords * num_blades ./ (2.0 * pi * radii)
    warnid = findall(x -> x > 0.3, solidity)
    if verbose && !isempty(warnid)
        @warn "Solidity greater than 0.3 at sections $(warnid)"
    end
    return solidity
end


check_solidity (generic function with 1 method)

# Run Script

In [6]:
# - Base Values - #
Vinf_base = 10.0 #m/s
thrust_base = 3.0 #N
exit_area_base = pi * (2.75 * 0.0254)^2 #m
Rtip_base = 2.5 * 0.0254 #m
Rhub_base = 0.25 * Rtip_base #m
radii_base = collect(range(Rhub_base, Rtip_base, 10)) #m
num_blades_base = 2
lift_polars_base = [[5.0 * pi/180.0 0.5; 6.0 * pi/180.0 0.7]]

chords, twists, debug = opt_prelim(
    Vinf_base,
    thrust_base,
    exit_area_base,
    Rtip_base,
    Rhub_base,
    radii_base,
    num_blades_base,
    lift_polars_base;
    rho=1.225, #kg/m3
    flow_coeff=0.4,
    ambient_static_pressure=101325.0, #Pa
    ambient_static_temperature=288.15, #K
    adiabatic_stage_efficiency=0.85,
    c_p=1005.0, #J/kg-K
    lift_coefficients=[0.6],
    verbose=true,
)

# get intermediate outputs
(;
    solidity,
    circulations,
    stagger_angles,
    angles_of_attack,
    swirl_velocities,
    inflow_angles,
    omega,
    change_in_tip_swirl,
    work_coeff,
    enthalpy_rise,
    temperature_ratio,
    pressure_ratio,
    exhaust_total_pressure,
    tip_rotational_velocity,
    fan_face_axial_velocity,
    fan_annulus_area,
    Vexit,
) = debug

println("Exit Area = ", exit_area_base, "m")
println("Exit Velocity = ", Vexit, "m/s")

println("Fan Area = ", fan_annulus_area, "m")
println("Fan Face Axial Velocity = ", fan_face_axial_velocity, "m/s")

println("Tip Rotational Velocity = ", tip_rotational_velocity, "m/s")
println("Omega = ", omega, "rad/s")
println("Change in Tip Swirl = ", change_in_tip_swirl, "m/s")

println("Work Coefficient = ", work_coeff)
println("Enthalpy Rise = ", enthalpy_rise, "J/kg")
println("Pressure Ratio = ", pressure_ratio)
println("Exhaust Total Pressure = ", exhaust_total_pressure, "Pa")
println("Temperature Ratio = ", temperature_ratio)

println("Stagger, degrees:")
display(stagger_angles*180/pi)

println("Inflow Angles, degrres:")
display(inflow_angles*180/pi)

println("Angle of Attack, degrees:")
display(angles_of_attack*180/pi)

println("Solidity:")
display(solidity)

println("Chords, m")
display(chords)

println("Swirl Velocities, m/s")
display(swirl_velocities)

println("Circulations, m^2/s")
display(circulations)

Exit Area = 0.015327901242699303

┌ Warning: Solidity greater than 0.3 at sections [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
└ @ Main In[5]:312


m
Exit Velocity = 18.59311084645179m/s
Fan Area = 0.011875956541347604m
Fan Face Axial Velocity = 23.997508399153773m/s
Tip Rotational Velocity = 59.99377099788443m/s
Omega = 944.7837952422746rad/s
Change in Tip Swirl = 2.4071342480992812m/s
Work Coefficient = 0.04012306957974294
Enthalpy Rise = 144.41306084163298J/kg
Pressure Ratio = 1.001484358674928
Exhaust Total Pressure = 101536.74355970592Pa
Temperature Ratio = 1.000498679812258
Stagger, degrees:


10-element Vector{Float64}:
 26.505383208083497
 34.3055710922652
 40.66913932790742
 45.84019174590991
 50.06101069119639
 53.53624346792648
 56.42751306414704
 58.85899417569471
 60.925293798087374
 62.698590513648185

Inflow Angles, degrres:


...

10-element Vector{Float64}:
 0.06539858737016367
 0.06595920281677818
 0.06622361526289311
 0.06636858618889414
 0.06645646054485917
 0.06651368152255986
 0.06655299755089866
 0.06658116288365079
 0.06660202507639015
 0.06661790562139945

Swirl Velocities, m/s


10-element Vector{Float64}:
 9.628536992397125
 7.221402744297844
 5.777122195438276
 4.8142684961985625
 4.126515853884483
 3.610701372148922
 3.209512330799042
 2.888561097719138
 2.625964634290125
 2.4071342480992812

Circulations, m^2/s


10-element Vector{Float64}:
 0.48020193964710134
 0.48020193964710145
 0.4802019396471014
 0.48020193964710134
 0.4802019396471014
 0.48020193964710145
 0.48020193964710145
 0.4802019396471014
 0.4802019396471014
 0.48020193964710134